<a href="https://colab.research.google.com/github/Aqeel151/Myproject/blob/master/Project_2_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
!pip install langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.3 MB/s eta 0:00:00


In [ ]:
import os



In [ ]:
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
from uuid import uuid4

In [ ]:
from tqdm import tqdm




In [ ]:
import time
from typing import List

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "new-online-rag-project"  # change if desired

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
vector = embeddings.embed_query("Hello world")

In [ ]:
vector[:10]

[0.04703257977962494,
 -0.04019005596637726,
 -0.02902696467936039,
 -0.026809632778167725,
 0.01892058178782463,
 -8.365760731976479e-05,
 0.03111639805138111,
 -0.019520554691553116,
 0.011491335928440094,
 0.009625785052776337]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [ ]:
document_1

Document(metadata={'source': 'tweet'}, page_content='I had chocalate chip pancakes and scrambled eggs for breakfast this morning.')

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4
uuid4()

UUID('f44cbe2c-ef9b-4367-8223-748ff6c23384')

In [ ]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['53ed9460-49b3-43dd-9ef6-818599ac3ff1',
 '7e8f01cf-4333-4ea3-a467-147d29e25d17',
 'e957d1ae-345d-4872-b40a-2dd6b533e516',
 '3e31a270-caf1-4618-80f4-a8ed6b615bf2',
 '0c5d6357-6257-479d-ae88-aab29ce97211',
 'f9657b2a-4142-4efa-9829-acddacafa9d1',
 'aa15be8d-762a-41d4-99ba-6209e607c75a',
 'fa7294f2-f560-40ec-aeb1-bb79bb801016',
 '50f6093b-70fc-47de-bfe6-73ef7297fd50',
 'fa9a52d0-b416-4f61-be54-c7c0ec5ee169']

In [ ]:
#Data Retrival
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",

)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
#Data Retrival

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


In [ ]:
#Data Retrival
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
#Data Retrival

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.577374] I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* [SIM=0.537337] I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]
* [SIM=0.533720] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
!pip install langchain langchain-google-genai


In [ ]:
import langchain_google_genai as genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
def answer_to_user(query: str):
  #vector search
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))
  #pass to Model Vector Result + User Query

  final_answer = llm.invoke(f"Answer this user Query: {query},Here are some references to answer{vector_results}")
 # ChatGoogleGenAI(results, query)
  return final_answer


In [ ]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

2


In [ ]:
answer.content

"LangChain simplifies working with Large Language Models (LLMs).  While the provided documents mention LangChain and another framework (LangGraph), they don't directly support the statement about LangChain's ease of use.  One document expresses excitement about a LangChain project, implying ease of use is possible, but doesn't explicitly confirm it.  More information is needed to definitively answer the query.\n"

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#/content/state_of_the_union.txt

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Get the absolute path to the directory containing this notebook
notebook_dir = os.path.dirname(os.path.abspath("/content/state_of_the_union.txt"))


# Construct the full path to your text file
file_path = os.path.join(notebook_dir, "state_of_the_union.txt")

# Use the full path when creating the TextLoader
loader = TextLoader(file_path)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Load the content of the file into the state_of_the_union variable
with open(file_path, 'r') as f:
    state_of_the_union = f.read()

# Now you can create documents using the state_of_the_union variable
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])
print(texts[1])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.'
page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.'


In [ ]:
with open('/content/state_of_the_union.txt', 'r') as f:
        state_of_the_union = f.read()

In [ ]:
from tqdm import tqdm

In [ ]:
import time

In [ ]:

# Initialize a list to hold the vectors to be upserted
vectors_to_upsert = []

# Create embeddings and prepare them for upsert
for i, doc in enumerate(tqdm(docs)):
    # Generate the embedding for the document content
    vector = embeddings.embed_query(doc.page_content)

    # Create a unique ID for each document chunk
    vector_id = f"{doc.metadata['source']}_chunk_{i}"

    # Prepare the tuple (ID, vector, metadata) for upsert
    vectors_to_upsert.append((vector_id, vector, {"text": doc.page_content}))

# Upsert all vectors to Pinecone in a single batch
index.upsert(vectors_to_upsert)
# Introduce a delay to avoid hitting rate limits
time.sleep(1) # Wait for 1 second

  0%|          | 0/91 [00:00<?, ?it/s]


GoogleGenerativeAIError: Error embedding content: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:558109086855'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "quota_location"
  value: "us-east7"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "consumer"
  value: "projects/558109086855"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]

In [ ]:
from langchain.vectorstores import Pinecone


In [ ]:
from langchain.vectorstores import Pinecone

retriever = Pinecone(pinecone_index=index, embedding_function=embeddings.embed_query, text_key="text")

<ipython-input-54-03ac373f3cd4>:3: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever = Pinecone(pinecone_index=index, embedding_function=embeddings.embed_query, text_key="text")


TypeError: Pinecone.__init__() got an unexpected keyword argument 'pinecone_index'

In [ ]:
from langchain.vectorstores import Pinecone

# Use index and embeddings directly as arguments
retriever = Pinecone(index, embeddings.embed_query, text_key="text")

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
!pip install -U langchain-google-genai # update or install the necessary langchain integration.
from langchain_google_genai import ChatGoogleGenerativeAI # Replace the import statement

In [ ]:
from langchain.chains import RetrievalQA


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from langchain.vectorstores import Pinecone

In [ ]:
# Assuming you have already initialized 'retriever' as shown in previous code

# Initialize the Gemini model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Now create the QA chain using the initialized gemini_model
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Input should be a valid dictionary or instance of BaseRetriever [type=model_type, input_value=<langchain_community.vect...bject at 0x7f697a7e9690>, input_type=Pinecone]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type